In [38]:
import pandas as pd
import numpy as np
import glob

# putting all csv files into one list
csv_files = [
    r"..\Data\ap_bus_details.csv",
    r"..\Data\Assam_bus_details.csv",
    r"..\Data\HimachalPradesh_bus_details.csv",
    r"..\Data\JammuKashmir_bus_details.csv",
    r"..\Data\Kadamba_bus_details.csv",
    r"..\Data\Kerala_bus_details.csv",
    r"..\Data\NorthBengal_bus_details.csv",
    r"..\Data\Punjab_bus_details.csv",
    r"..\Data\Rajasthan_bus_details.csv",
    r"..\Data\SouthBengal_bus_details.csv",
    r"..\Data\Telangana_bus_details.csv",
    r"..\Data\UttaraPradesh_bus_details.csv",
    r"..\Data\WestBengal_bus_details.csv"
]
df = [pd.read_csv(file) for file in csv_files]

# combine all dataframes in the list
combine_df = pd.concat(df, ignore_index=True)
combine_df.to_csv("Buses_data.csv", index=False)

df = pd.read_csv(r"..\Data\Buses_data.csv")
df = df.dropna() # drop null values

#add id column
id_column = pd.Series(range(1, len(df) + 1), name='id')
df = pd.concat([id_column, df], axis=1)


# replace string to empty string
df['Price'] = df['Price'].str.replace('INR ', '', regex=False)
df['Price'] = df['Price'].astype(np.float64)

#extract the digits(0-9)
df['Seat_Availability'] = df['Seat_Availability'].str.extract(r'(\d+)')
df['Seat_Availability'] = df['Seat_Availability'].astype(np.int64)

# Convert all numeric columns to Python native types (int and float)
df = df.applymap(lambda x: float(x) if isinstance(x, (int, np.int64, np.float64)) else x)
df = df.applymap(lambda x: int(x) if isinstance(x, np.int64) else x)

from mysql import connector
connection = connector.connect(
    host = "localhost",
    user = "root",
    password = "root",
    autocommit = True
)
my_cursor = connection.cursor()
query = "create database if not exists com_Guvi_RedBus"
my_cursor.execute(query)
query = "use com_guvi_redbus"
my_cursor.execute(query)

a = ",".join(f"{i} {j}"
             for i,j in zip(df.columns,df.dtypes)).replace("float64","FLOAT").replace("object","TEXT").replace("int64","INT") #replace dtypes from pandas to mysql
table_name = "Bus_details"
my_cursor.execute(f"create table if not exists {table_name} ({a})")

# Ensure correct number of values when inserting (including 'id' column)
columns = df.columns.tolist()  # Get all column names
columns_str = ', '.join(columns)  # Convert columns to a comma-separated string
placeholders = ', '.join(['%s'] * len(columns))  # Create placeholders for the number of columns

# Insert data into table
data_to_insert = [tuple(row) for row in df.values]  # Convert dataframe rows to tuples for batch insert

# Use executemany for efficient bulk insertion
my_cursor.executemany(f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})", data_to_insert)

C:\Users\vmami\AppData\Local\Temp\ipykernel_1736\1969020623.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: float(x) if isinstance(x, (int, np.int64, np.float64)) else x)
C:\Users\vmami\AppData\Local\Temp\ipykernel_1736\1969020623.py:45: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: int(x) if isinstance(x, np.int64) else x)


In [42]:
query = "describe Bus_details"
my_cursor.execute(query)
for db in my_cursor:
    print(db)

('id', 'int', 'NO', 'PRI', None, '')
('Route_Name', 'text', 'YES', '', None, '')
('Route_Link', 'text', 'YES', '', None, '')
('Bus_Name', 'text', 'YES', '', None, '')
('Bus_Type', 'text', 'YES', '', None, '')
('Departing_Time', 'text', 'YES', '', None, '')
('Duration', 'text', 'YES', '', None, '')
('Reaching_Time', 'text', 'YES', '', None, '')
('Star_Rating', 'float', 'YES', '', None, '')
('Price', 'float', 'YES', '', None, '')
('Seat_Availability', 'int', 'YES', '', None, '')
